### Used to convert pydantic schema definitions for data types into json objects used by react to generate the interface for forms

### !!! Only works inside monorepo and not a containerized api instance !!!

In [5]:
import sys
sys.path.append('..')
!{sys.executable} -m pip install jsonref

from pydantic_models import dataset_schema
from pydantic_models import datafeed_schema
from pydantic_models import anomaly_schema
from pydantic_models import detector_schema
from pydantic_models import prediction_schema
from pydantic_models import health_schema

from pathlib import Path
import json
import jsonref
import datetime
import copy

In [6]:
options={}
options["Datafeed"]=datafeed_schema.DatafeedPayload
options["Dataset"]=dataset_schema.Dataset
options["Anomaly"]=anomaly_schema.AnomalyPayload
options["Detector"]=detector_schema.DetectorPayload
options["Prediction"]=prediction_schema.PredictionPayload
options["Health"]=health_schema.Health

PATH_TO_REACT_PROJECT="../../apps/time-series/src/"

In [7]:
anomaly=jsonref.replace_refs(anomaly_schema.Anomaly.model_json_schema())
anomaly_payload=jsonref.replace_refs(anomaly_schema.AnomalyPayload.model_json_schema())

In [8]:
print(json.dumps(anomaly, indent=2))

{
  "description": "Anomaly model.",
  "properties": {
    "id": {
      "format": "uuid",
      "title": "Id",
      "type": "string"
    },
    "time_created": {
      "format": "date-time",
      "title": "Time Created",
      "type": "string"
    },
    "time_updated": {
      "format": "date-time",
      "title": "Time Updated",
      "type": "string"
    },
    "value": {
      "title": "Value",
      "type": "number"
    },
    "time": {
      "format": "date-time",
      "title": "Time",
      "type": "string"
    },
    "anomaly_score": {
      "title": "Anomaly Score",
      "type": "number"
    },
    "threshold": {
      "title": "Threshold",
      "type": "number"
    },
    "status": {
      "title": "Status",
      "type": "string"
    },
    "severity": {
      "title": "Severity",
      "type": "string"
    },
    "tags": {
      "data_type": "list",
      "default": [],
      "items": {
        "type": "string"
      },
      "title": "Tags",
      "type": "array"
   

In [10]:
print(json.dumps(anomaly_payload, indent=2))

{
  "$defs": {
    "Severity": {
      "enum": [
        "low",
        "medium",
        "high"
      ],
      "title": "Severity",
      "type": "string"
    }
  },
  "description": "Anomaly payload model.",
  "properties": {
    "value": {
      "data_type": "number",
      "default": 0,
      "title": "Value",
      "type": "number"
    },
    "time": {
      "data_type": "datetime",
      "default": "2024-10-09T13:23:07.053186",
      "description": "The datetime of the record labeled anomalous",
      "format": "date-time",
      "title": "Time",
      "type": "string"
    },
    "anomaly_score": {
      "data_type": "number",
      "default": 0,
      "title": "Anomaly Score",
      "type": "number"
    },
    "threshold": {
      "data_type": "number",
      "default": 0,
      "title": "Threshold",
      "type": "number"
    },
    "status": {
      "data_type": "text",
      "default": "test",
      "maxLength": 127,
      "minLength": 1,
      "title": "Status",
      "type"

In [12]:
def load_pydantic_model(SELECTION):
    model_schema=jsonref.replace_refs(options[SELECTION].model_json_schema()) #resolves references to enums
    properties=model_schema["properties"]
    return properties

def getModelAttributes(properties):
    modelAttributes=[]

    for item in properties: #item is the name of each attribute
        entry={}

        entry["name"]=item

        if 'data_type' in properties[item]: #only add properties with a type value
            entry["data_type"]=properties[item]["data_type"]
            entry["editable"]=True
            entry["stats"]=True
           
            if 'default' in properties[item]:
                entry["default"]=properties[item]['default']

            defineOptions(entry,properties[item])
            defineEmptyFilter(entry,properties[item])
            defineFilterRule(entry,properties[item])
            modelAttributes.append(entry)

    return modelAttributes

def defineOptions(entry,property):
     if property["data_type"] =='select':
        options=[]
        for enum_item in property["allOf"][0]["enum"]:
            options.append({"value":enum_item, "label":enum_item})
        entry["options"]=options
     if property["data_type"] =='list':
        entry["options"]=[]

def defineEmptyFilter(entry,property):    
    if property["data_type"] =='text' or property["data_type"]=="number" or property["data_type"]=="boolean" or property["data_type"]=="id":
         entry["filter_empty"]=""
    if property["data_type"] =='select' or property["data_type"] =='list':
        entry["filter_empty"]=[]
    if property["data_type"] =='datetime':
        entry["filter_empty"]={"timespan_begin":"","timespan_end":""}

def defineFilterRule(entry,property):    
    if property["data_type"] =='text':
         entry["filter_rule"]="equals"
    if property["data_type"] =='select' or property["data_type"] =='list':
        entry["filter_rule"]="any"
    if property["data_type"] =='number':
        entry["filter_rule"]=">"

def add_base_fields(model_attributes):
    for item in ["time_created","time_updated"]:
        entry={}
        entry["name"]=item
        entry["data_type"]="datetime"
        entry["filter_empty"]={"timespan_begin":"","timespan_end":""}
        entry["default"]=datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S.%f%z')
        model_attributes.append(entry)

    for item in ["id"]:
        entry={}
        entry["name"]=item
        entry["data_type"]="id"
        entry["editable"]=False
        entry["stats"]=False
        entry["filter_empty"]=""
        model_attributes.append(entry)

def load_json_file(model):
    path=Path(PATH_TO_REACT_PROJECT) / "metadata" / str(model+"ModelData.json")
    f = open(path)
    json_data=json.load(f)
    return json_data

def save_json_file(model,json_data):
    path=Path(PATH_TO_REACT_PROJECT) / "metadata" / str(model+"ModelData.json")
    with open(path,"w") as json_file:
        json.dump(json_data,json_file)

In [14]:
SELECTION="Dataset"
#SELECTION="Datafeed"
#SELECTION="Anomaly"
#SELECTION="Detector"
#SELECTION="Prediction"

#data_models=["Dataset","Datafeed","Anomaly","Detector","Prediction","Health"]

for model in data_models:
    properties=load_pydantic_model(model)

    json_data=load_json_file(model)
    json_data["model_attributes"]=getModelAttributes(properties)

    add_base_fields(json_data["model_attributes"])

    save_json_file(model,json_data)